In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 5.3 MB 23.9 MB/s 
     |████████████████████████████████| 1.3 MB 42.1 MB/s 
     |████████████████████████████████| 163 kB 57.9 MB/s 
     |████████████████████████████████| 7.6 MB 51.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=03fa7df7ff6e0374d53d493cfc1ba75e916a2f1e86e6c0117119b95018d4e6a9
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import InputExample, SentenceTransformer, models, losses, evaluation, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from scipy import spatial
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from data_processing import get_data

In [3]:
x_train, x_test1, x_test2, x_test3, x_test4, x_test5, y_train, y_test1, y_test2, y_test3, y_test4, y_test5 = get_data()
def convert_list_data_into_InputExample(x, y):
    examples = []
    for s, label in zip(x, y):
        question1, question2 = s
        examples.append(InputExample(texts=[question1, question2], label=float(label)))
    return examples

train = convert_list_data_into_InputExample(x_train, y_train)
test1 = convert_list_data_into_InputExample(x_test1, y_test1)
test2 = convert_list_data_into_InputExample(x_test2, y_test2)
test3 = convert_list_data_into_InputExample(x_test3, y_test3)
test4 = convert_list_data_into_InputExample(x_test4, y_test4)
test5 = convert_list_data_into_InputExample(x_test5, y_test5)

In [4]:
vectorizer = CountVectorizer()
sample = np.array(x_test2)
sample = sample.tolist()

def f(number):
    return float(number)

In [5]:
from spacy import displacy
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
def get_noun(s: str):
    doc = nlp(s)
    if doc[0].pos_ == 'NOUN':
        return 1
    else:
        return 0

In [10]:
def change_noun_weight(sample):
    n = len(sample)
    simi = []
    for i in range(n):
        fit = vectorizer.fit_transform(sample[i]).todense()
        vector1 = fit[0].tolist()[0]
        vector2 = fit[1].tolist()[0]
        float_numbers_iterator1 = map(f, vector1)
        float_numbers_iterator2 = map(f, vector2)
        vector1 = list(float_numbers_iterator1)
        vector2 = list(float_numbers_iterator2)
        words = vectorizer.get_feature_names_out()
        for j in range(len(words)):
            pro = get_noun(words[j])
            if pro==0:
                vector1[j]=0.0
                vector2[j]=0.0
        similarity = util.cos_sim(vector1, vector2)
        simi.append(similarity)
    return simi

In [8]:
def f1_score(simi, y):
    TP = 0
    FP = 0
    FN = 0
    prediction_label = [0]*len(simi)
    for i in range(len(simi)):
        if simi[i] > 0.5:
            prediction_label[i]=1
            if y[i]=='1':
                TP+=1
            elif y[i]=='0':
                FP+=1
        else:
            if y[i]=='1':
                FN+=1
    R=TP/(TP+FN)
    P=TP/(TP+FP)
    f1_score1=2*P*R/(P+R)
    return f1_score1

In [11]:
simi_list= change_noun_weight(sample)
f = f1_score(simi_list, y_test2)
print(f)

0.563816604708798
